In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
store = pd.HDFStore('store.h5')

In [ ]:
tr = pd.read_csv('../input/X_train.csv')
te = pd.read_csv('../input/X_test.csv')

In [ ]:
target = pd.read_csv('../input/y_train.csv')
ss = pd.read_csv('../input/sample_submission.csv')

In [ ]:
# It looks like linear acceleration in Z direction has gravity recorded. Lets remove gravity part.
import scipy.constants as const
tr['linear_acceleration_Z'] = tr['linear_acceleration_Z'] + const.g

In [ ]:
tr.shape, te.shape

In [ ]:
# Number of time series in train and test sets
tr.shape[0]/128, te.shape[0]/128

In [ ]:
tr.head()

In [ ]:
sns.countplot(y = 'surface', data = target)
plt.show()

In [ ]:
# Function to convert quaternions to euler angles
def t2(x):
    if x > +1.0:
        return +1.0
    elif x < -1.0:
        return -1.0
    else:
        return x

vt2 = np.vectorize(t2)

def quaternion_to_euler(x, y, z, w):
    
    t0 = +2.0 * ((w * x) + (y * z))
    t1 = +1.0 - 2.0 * ((x**2) + (y**2))
    X = np.arctan2(t0, t1)

    t2 = +2.0 * (w * y - z * x)
    t2 = vt2(t2)
    Y = np.arcsin(t2)

    t3 = +2.0 * (w * z + x * y)
    t4 = +1.0 - 2.0 * (y**2 + z**2)
    Z = np.arctan2(t3, t4)

    return X, Y, Z

In [ ]:
# Converting to Euler Angles
def add_euler_angles(actual):
    
    x, y, z, w = actual['orientation_X'].values, actual['orientation_Y'].values, actual['orientation_Z'].values, actual['orientation_W'].values
    nx,ny,nz = quaternion_to_euler(x, y, z, w)
       
    actual['euler_x'] = nx
    actual['euler_y'] = ny
    actual['euler_z'] = nz
    
    return actual

In [ ]:
# Applying add_euler_angles to dataset
tr = add_euler_angles(tr)
te = add_euler_angles(te)

In [ ]:
tr.head()

In [ ]:
# Checkpoint 1 - Saving data
store['tr_1'] = tr
store['te_1'] = te

In [ ]:
def integrate(actual, col, new_value):
    temp = pd.DataFrame()
    for x in range(int(actual.shape[0]/128)):
        temp = pd.concat([temp, np.cumsum(actual.loc[actual['series_id']==x,col])], axis=0)
    temp.reset_index(inplace=True)
    temp.drop('index', axis=1, inplace=True)
    temp.columns = [new_value + x[-1] for x in col]
    return pd.concat([actual, temp], axis=1)

In [ ]:
tr = integrate(tr, col=['linear_acceleration_X','linear_acceleration_Y','linear_acceleration_Z'], new_value='linear_velocity_')
te = integrate(te, col=['linear_acceleration_X','linear_acceleration_Y','linear_acceleration_Z'], new_value='linear_velocity_')

In [ ]:
tr = integrate(tr, col=['linear_velocity_X','linear_velocity_Y','linear_velocity_Z'], new_value='linear_disp_')
te = integrate(te, col=['linear_velocity_X','linear_velocity_Y','linear_velocity_Z'], new_value='linear_disp_')

In [ ]:
tr.head()

In [ ]:
# Feature Engineering - Physics
def fe_ph(actual):
    
    # Total rotational kinetic energy KE - 0.5 * I * omega**2 where I = m * r**2 assuming m=2 and r=1
    actual['total_rot_ke'] = (actual['angular_velocity_X'] ** 2 + 
                              actual['angular_velocity_Y'] ** 2 + 
                              actual['angular_velocity_Z'] ** 2)
    
    # Total linear accelration = root of sum of squares of accelearation in x, y direction
    actual['total_lin_accel'] = (actual['linear_acceleration_X'] ** 2 + 
                                 actual['linear_acceleration_Y'] ** 2) ** 0.5
    
    
    actual['ke_vs_accel'] = actual['total_rot_ke'] / actual['total_lin_accel']
    
    
    # Total Angle - nothing like this exists in reality
    actual['total_angle'] = (actual['euler_x'] ** 2 + 
                             actual['euler_y'] ** 2 + 
                             actual['euler_z'] ** 2) ** 5
    
    actual['angle_vs_acc'] = actual['total_angle'] / actual['total_lin_accel']
    actual['angle_vs_vel'] = actual['total_angle'] / actual['total_rot_ke']
    
    return actual

In [ ]:
tr = fe_ph(tr)
te = fe_ph(te)

In [ ]:
# Checkpoint 2 - Saving data
store['tr_2'] = tr
store['te_2'] = te

In [ ]:
tr = store['tr_2']
te = store['te_2']

In [ ]:
# Change of Change calc function - 2nd Order Derivative
def f1(x):
    return np.mean(np.diff(np.abs(np.diff(x))))
    
#  Change calc function - 1st order Derivative
def f2(x):
    return np.mean(np.abs(np.diff(x)))

def slope(x):
    

In [ ]:
# Feature Engineering - Statistics
# Capturing the essense of time series in to statistical features.

def fe(actual):
    new = pd.DataFrame()

    # Mean, Min, Max, Std and Max/Min Calculations.
    for col in actual.columns:
        if col in ['row_id', 'series_id', 'measurement_number']:
            continue
        new[col + '_mean'] = actual.groupby(['series_id'])[col].mean()
        new[col + '_min'] = actual.groupby(['series_id'])[col].min()
        new[col + '_max'] = actual.groupby(['series_id'])[col].max()
        new[col + '_std'] = actual.groupby(['series_id'])[col].std()
        new[col + '_max_to_min'] = new[col + '_max'] / new[col + '_min']
        
        # Change. 1st order.
        new[col + '_mean_abs_change'] = actual.groupby('series_id')[col].apply(f2)
        
        # Change of Change. 2nd order.
        new[col + '_mean_change_of_abs_change'] = actual.groupby('series_id')[col].apply(f1)
        
        # Abslute max and min
        new[col + '_abs_max'] = actual.groupby('series_id')[col].apply(lambda x: np.max(np.abs(x)))
        new[col + '_abs_min'] = actual.groupby('series_id')[col].apply(lambda x: np.min(np.abs(x)))

    return new

In [ ]:
import time
start = time.time()
tr = fe(tr)
te = fe(te)
print("Complete in %s seconds" % (time.time()-start))

In [ ]:
tr.head()

In [ ]:
# Checkpoint 3
store['tr_3'] = tr
store['te_3'] = te
store.close()

In [ ]:
drops = ['linear_velocity_X_min','linear_velocity_X_max_to_min','linear_velocity_Y_min','linear_velocity_Y_max_to_min','linear_velocity_Z_min','linear_velocity_Z_max_to_min',
        'linear_velocity_X_mean_abs_change','linear_velocity_Y_mean_abs_change','linear_velocity_Z_mean_abs_change',
        'linear_velocity_X_mean_change_of_abs_change','linear_velocity_Y_mean_change_of_abs_change','linear_velocity_Z_mean_change_of_abs_change']

In [ ]:
tr.drop(drops, axis=1, inplace=True)
te.drop(drops, axis=1, inplace=True)

In [ ]:
# Now we Train the Model
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
target['surface'] = le.fit_transform(target['surface'])

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, StratifiedKFold
folds = StratifiedKFold(n_splits=21, shuffle=True, random_state=89)
sub_preds_rf = np.zeros((te.shape[0], 9))
oof_preds_rf = np.zeros((tr.shape[0]))
score = 0
for i, (train_index, test_index) in enumerate(folds.split(tr, target['surface'])):
    print('-'*20, i, '-'*20)
    
    clf =  RandomForestClassifier(n_estimators = 200, n_jobs = -1)
    clf.fit(tr.iloc[train_index], target['surface'][train_index])
    oof_preds_rf[test_index] = clf.predict(tr.iloc[test_index])
    sub_preds_rf += clf.predict_proba(te) / folds.n_splits
    score += clf.score(tr.iloc[test_index], target['surface'][test_index])
    print('score ', clf.score(tr.iloc[test_index], target['surface'][test_index]))
    importances = clf.feature_importances_
    indices = np.argsort(importances)
    features = tr.columns

    hm = 30
    plt.figure(figsize=(7, 10))
    plt.title('Feature Importances')
    plt.barh(range(len(indices[:hm])), importances[indices][:hm], color='b', align='center')
    plt.yticks(range(len(indices[:hm])), [features[i] for i in indices])
    plt.xlabel('Relative Importance')
    plt.show()

print('Avg Accuracy', score / folds.n_splits)

In [ ]:
import itertools

def plot_confusion_matrix(truth, pred, classes, normalize=False, title=''):
    cm = confusion_matrix(truth, pred)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    
    plt.figure(figsize=(10, 10))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Confusion matrix', size=15)
    plt.colorbar(fraction=0.046, pad=0.04)
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.grid(False)
    plt.tight_layout()

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
plot_confusion_matrix(target['surface'], oof_preds_rf, le.classes_)

In [ ]:
ss['surface'] = le.inverse_transform(sub_preds_rf.argmax(axis=1))
ss.to_csv('rf.csv', index=False)
ss.head(10)